# EPAM

In [42]:
import re
import warnings
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup

# Switch warning off for `bs4`.
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [2]:
posts_file = 'NLP_dataset_final/Posts.csv'

In [3]:
%time posts_df = pd.read_csv(posts_file, encoding='ISO-8859-1')

CPU times: user 12.5 s, sys: 998 ms, total: 13.5 s
Wall time: 14 s


<string>:2: DtypeWarning: Columns (3,18,19) have mixed types. Specify dtype option on import or set low_memory=False.


In [44]:
def replace_unescape(text, unescape_dict={'&lt;': '<', '&gt;': '>', '&amp;': '&', '\r': ''}):
    for k, v in unescape_dict.items():
        text = text.replace(k, v)
    for k, v in unescape_dict.items():
        text = text.replace(k, v)
    return text


def delete_tag(text, tags):
    for tag in tags:
        text = text.replace('<{}>'.format(tag), '')
        text = text.replace('</{}>'.format(tag), '')
    return text


def get_text(text, sep='\n', delete_code=True, code_replacer='CODE', tags=['p', 'pre']):
    # Code.
    soup = BeautifulSoup(text, 'lxml')
    if delete_code:
        for i, tag in enumerate(soup.find_all('code')):
            tag.replaceWith(code_replacer.format(i))
    
    # Main part.
    return sep.join(
        delete_tag(replace_unescape(p_tag.text), tags) 
        for p_tag in soup.find_all(tags)
    )

In [52]:
sample = posts_df.Body[310581]
sample

'<p>You\'re using the shorter form of lambda expressions, which only allow a single expressions.<br>\r\r\r\nYou need to the long form, which allows multiple statements.</p>\r\r\r\n\r\r\r\n<p>For example:</p>\r\r\r\n\r\r\r\n<pre><code>items.ForEach(item =&gt; {\r\r\r\n    if (item.Contains("I Care About")) \r\r\r\n        whatICareAbout += item + ", ";\r\r\r\n});\r\r\r\n</code></pre>\r\r\r\n'

In [53]:
get_text(sample)

"You're using the shorter form of lambda expressions, which only allow a single expressions.\nYou need to the long form, which allows multiple statements.\nFor example:\nCODE"

In [43]:
clean_bodies = []
for body in tqdm(posts_df.Body):
    clean_bodies.append(get_text(body))

posts_df['CleanBody'] = clean_bodies

100%|██████████| 740551/740551 [08:03<00:00, 1532.75it/s]
